# Train a few versions of the tuneable model

Build a few versions of the classification layers to get a feel of what parameters should be used for the hyperparameter optimalization. TensorFlow 2 is required to use Keras-Tuner, which needs to be installed in our Google colab instance beforehand.

In [1]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

!pip install keras-tuner

     |████████████████████████████████| 86.3MB 121kB/s 
     |████████████████████████████████| 3.8MB 41.5MB/s 
     |████████████████████████████████| 450kB 54.0MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.8.2 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 92kB 6.9MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=9fa666e23319a87520e2249b01fed78c704a31c530bb3ebdece189912b66e72b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


*NOTE*: The runtime needs to be restarted after updating TensorFlow.

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

## 0. Prepare data

Mount my drive and load the bottleneck features.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
bottleNeck_loaded = np.load('/content/drive/My Drive/DeepStep_colab/deepStep_bottleNeck_data.npz')

train_bottleNeck = bottleNeck_loaded['train_bottleNeck']
train_labels = bottleNeck_loaded['train_labels']
validate_bottleNeck = bottleNeck_loaded['validate_bottleNeck']
validate_labels = bottleNeck_loaded['validate_labels']

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 1. Define the trainable models

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D, InputLayer, ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD

Define swish as a possible activation function:

In [0]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

class Swish(Layer):
    def __init__(self, beta, **kwargs):
        super(Swish, self).__init__(**kwargs)
        self.beta = K.cast_to_floatx(beta)

    def call(self, inputs):
        return K.sigmoid(self.beta * inputs) * inputs

    def get_config(self):
        config = {'beta': float(self.beta)}
        base_config = super(Swish, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

Define the model and the optimizable parameters.

In [0]:
def build_class_model(hp):
    class_model = Sequential()
    class_model.add(InputLayer(input_shape = (6, 6, 2048)))

    pooling = hp.Choice('Pooling', values = ['None', 'Max', 'Avg'], ordered = False)
    if pooling == 'Max':
        class_model.add(GlobalMaxPooling2D())
    elif pooling == 'Avg':
        class_model.add(GlobalAveragePooling2D())

    class_model.add(Flatten())

    activation = hp.Choice('Activation', values = ['ReLU', 'LeakyReLU', 'Swish'], ordered = False)
    if activation == 'ReLU':
        activation = ReLU()
    elif activation == 'LeakyReLU':
        activation = LeakyReLU()
    elif activation == 'Swish':
        activation = Swish(beta=0.3)

    regu = hp.Choice('Regularization', values = ['Dropout', 'BatchNorm'], ordered = False)
    

    for i in range(hp.Int('nLayers',min_value = 1, max_value = 3)):
        class_model.add(Dense(units=hp.Choice('nNeurons_' + str(i), values = [8, 16, 32, 64, 128, 256, 512])))

        if regu == 'BatchNorm':
            class_model.add(BatchNormalization())

        class_model.add(activation)

        if regu == 'Dropout':
            class_model.add(Dropout(hp.Float('Dropout_rate', min_value = 0.1, max_value = 0.5, step = 0.5, parent_name = 'Regularization', parent_values = ['Dropout'])))

    class_model.add(Dense(units = 6, activation = 'softmax'))

    lr = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    optimizer = hp.Choice('Optimizer', values = ['Adam', 'SGD'], ordered = False)

    if optimizer == 'Adam': 
        class_model.compile(optimzer = Adam(lr = lr), loss='categorical_crossentropy', metrics=['accuracy'])
    elif optimizer == 'SGD':
        class_model.compile(optimzer = SGD(lr = lr), loss='categorical_crossentropy', metrics=['accuracy'])

    return class_model

Subclass the Hyperband tuner to add support for optimizing batch size.

In [0]:
from kerastuner.tuners import Hyperband

In [0]:
class HyperbandBatch(Hyperband):
    def run_trial(self, trial, *fit_args, **fit_kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        fit_kwargs['batch_size'] = trial.hyperparameters.Choice('batch_size', values = [4, 8, 16, 32, 64, 128, 256])
        super(HyperbandBatch, self).run_trial(trial, *fit_args, **fit_kwargs)

Finally, build the tuner.

In [0]:
tuner = HyperbandBatch(
    build_class_model,
    objective='val_loss',
    max_epochs=200,
    factor=3,
    executions_per_trial=3,
    directory='/content',
    project_name='hyperopt_logs')

Summarize the search space.

In [9]:
tuner.search_space_summary()

## 2. Perform the search

Define an early stopping callback, to not use the maximum 100 epochs if it's not needed.

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights = True, verbose = 1)

Start the search.

In [11]:
tuner.search(train_bottleNeck, train_labels,
             shuffle = True,
             validation_data = [validate_bottleNeck, validate_labels],
             callbacks = [early_stopping])

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 3s 7ms/sample - loss: 1.0082 - accuracy: 0.7676 - val_loss: 0.5566 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 2s 3ms/sample - loss: 0.7406 - accuracy: 0.8926 - val_loss: 0.4376 - val_accuracy: 0.9688
Epoch 3/3
512/512 [==============================] - 2s 3ms/sample - loss: 0.5904 - accuracy: 0.9102 - val_loss: 0.3400 - val_accuracy: 0.9688
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 2s 5ms/sample - loss: 1.1142 - accuracy: 0.6816 - val_loss: 0.6240 - val_accuracy: 0.9375
Epoch 2/3
512/512 [==============================] - 1s 3ms/sample - loss: 0.8652 - accuracy: 0.8516 - val_loss: 0.5767 - val_accuracy: 0.9688
Epoch 3/3
512/512 [==============================] - 1s 3ms/sample - loss: 0.7095 - accuracy: 0.8867 - val_loss: 0.4503 - val_accuracy: 0.9844
Train on 512 samples, validate on 64 samples
Epoch 1

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 1ms/sample - loss: 1.5341 - accuracy: 0.3984 - val_loss: 1.3134 - val_accuracy: 0.5000
Epoch 2/3
512/512 [==============================] - 0s 448us/sample - loss: 1.1568 - accuracy: 0.6270 - val_loss: 1.0356 - val_accuracy: 0.6875
Epoch 3/3
512/512 [==============================] - 0s 399us/sample - loss: 1.0084 - accuracy: 0.6387 - val_loss: 0.8963 - val_accuracy: 0.7031
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 1ms/sample - loss: 1.4628 - accuracy: 0.4922 - val_loss: 1.2411 - val_accuracy: 0.5781
Epoch 2/3
512/512 [==============================] - 0s 471us/sample - loss: 1.1583 - accuracy: 0.6328 - val_loss: 1.0337 - val_accuracy: 0.7031
Epoch 3/3
512/512 [==============================] - 0s 433us/sample - loss: 1.0108 - accuracy: 0.6758 - val_loss: 0.9032 - val_accuracy: 0.7031
Train on 512 samples, validate on 64 samples

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.9734 - accuracy: 0.6582 - val_loss: 4.1975 - val_accuracy: 0.3125
Epoch 2/3
512/512 [==============================] - 0s 392us/sample - loss: 0.6328 - accuracy: 0.7832 - val_loss: 2.5892 - val_accuracy: 0.2969
Epoch 3/3
512/512 [==============================] - 0s 381us/sample - loss: 0.5417 - accuracy: 0.7793 - val_loss: 2.4171 - val_accuracy: 0.3438
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 1ms/sample - loss: 0.9303 - accuracy: 0.6973 - val_loss: 3.6338 - val_accuracy: 0.2812
Epoch 2/3
512/512 [==============================] - 0s 315us/sample - loss: 0.6188 - accuracy: 0.8105 - val_loss: 3.3110 - val_accuracy: 0.3281
Epoch 3/3
512/512 [==============================] - 0s 294us/sample - loss: 0.5131 - accuracy: 0.8125 - val_loss: 3.2814 - val_accuracy: 0.3281
Train on 512 samples, validate on 64 samples

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 2s 3ms/sample - loss: 1.1547 - accuracy: 0.6035 - val_loss: 1.0899 - val_accuracy: 0.6406
Epoch 2/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.9094 - accuracy: 0.6738 - val_loss: 1.0896 - val_accuracy: 0.4375
Epoch 3/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.7521 - accuracy: 0.7207 - val_loss: 0.9160 - val_accuracy: 0.6094
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 2s 3ms/sample - loss: 1.1356 - accuracy: 0.5898 - val_loss: 1.0866 - val_accuracy: 0.6094
Epoch 2/3
512/512 [==============================] - 1s 1ms/sample - loss: 0.9109 - accuracy: 0.6582 - val_loss: 1.0648 - val_accuracy: 0.4688
Epoch 3/3
512/512 [==============================] - 1s 1ms/sample - loss: 0.8254 - accuracy: 0.6953 - val_loss: 1.1106 - val_accuracy: 0.4219
Train on 512 samples, validate on 64 samples
Epoch 1

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 3ms/sample - loss: 0.3085 - accuracy: 0.9141 - val_loss: 0.3366 - val_accuracy: 0.9531
Epoch 2/3
512/512 [==============================] - 1s 994us/sample - loss: 0.0380 - accuracy: 0.9941 - val_loss: 0.2672 - val_accuracy: 0.9688
Epoch 3/3
512/512 [==============================] - 0s 878us/sample - loss: 0.0137 - accuracy: 0.9980 - val_loss: 0.2039 - val_accuracy: 0.9844
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 3ms/sample - loss: 0.3034 - accuracy: 0.9336 - val_loss: 0.2019 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 1s 1ms/sample - loss: 0.0316 - accuracy: 0.9980 - val_loss: 0.1821 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 856us/sample - loss: 0.0229 - accuracy: 0.9980 - val_loss: 0.1501 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 samples
E

...

Train on 512 samples, validate on 64 samples
Epoch 9/23
512/512 [==============================] - 1s 2ms/sample - loss: 0.5694 - accuracy: 0.7832 - val_loss: 0.1042 - val_accuracy: 0.9844
Epoch 10/23
512/512 [==============================] - 0s 408us/sample - loss: 0.1114 - accuracy: 0.9727 - val_loss: 0.0691 - val_accuracy: 0.9844
Epoch 11/23
512/512 [==============================] - 0s 422us/sample - loss: 0.1026 - accuracy: 0.9648 - val_loss: 0.0175 - val_accuracy: 1.0000
Epoch 12/23
512/512 [==============================] - 0s 433us/sample - loss: 0.1452 - accuracy: 0.9570 - val_loss: 0.0132 - val_accuracy: 1.0000
Epoch 13/23
512/512 [==============================] - 0s 386us/sample - loss: 0.0971 - accuracy: 0.9590 - val_loss: 0.0323 - val_accuracy: 0.9844
Epoch 14/23
512/512 [==============================] - 0s 343us/sample - loss: 0.0757 - accuracy: 0.9785 - val_loss: 0.0599 - val_accuracy: 0.9844
Epoch 15/23
512/512 [==============================] - 0s 372us/sample - los

Train on 512 samples, validate on 64 samples
Epoch 9/23
512/512 [==============================] - 1s 1ms/sample - loss: 0.4083 - accuracy: 0.8262 - val_loss: 0.0260 - val_accuracy: 1.0000
Epoch 10/23
512/512 [==============================] - 0s 377us/sample - loss: 0.0190 - accuracy: 0.9961 - val_loss: 0.0169 - val_accuracy: 1.0000
Epoch 11/23
512/512 [==============================] - 0s 396us/sample - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0108 - val_accuracy: 1.0000
Epoch 12/23
512/512 [==============================] - 0s 325us/sample - loss: 0.0147 - accuracy: 0.9961 - val_loss: 0.0162 - val_accuracy: 0.9844
Epoch 13/23
512/512 [==============================] - 0s 311us/sample - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0302 - val_accuracy: 0.9844
Epoch 14/23
512/512 [==============================] - 0s 358us/sample - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 15/23
512/512 [==============================] - 0s 361us/sample - los

Train on 512 samples, validate on 64 samples
Epoch 9/23
512/512 [==============================] - 3s 6ms/sample - loss: 0.3150 - accuracy: 0.8984 - val_loss: 0.0556 - val_accuracy: 0.9844
Epoch 10/23
512/512 [==============================] - 3s 5ms/sample - loss: 0.2272 - accuracy: 0.9180 - val_loss: 0.0552 - val_accuracy: 0.9844
Epoch 11/23
512/512 [==============================] - 3s 5ms/sample - loss: 0.2219 - accuracy: 0.9395 - val_loss: 0.0475 - val_accuracy: 0.9844
Epoch 12/23
512/512 [==============================] - 3s 5ms/sample - loss: 0.1898 - accuracy: 0.9512 - val_loss: 0.0397 - val_accuracy: 0.9844
Epoch 13/23
512/512 [==============================] - 2s 4ms/sample - loss: 0.1518 - accuracy: 0.9629 - val_loss: 0.0476 - val_accuracy: 0.9844
Epoch 14/23
512/512 [==============================] - 3s 6ms/sample - loss: 0.1934 - accuracy: 0.9512 - val_loss: 0.0283 - val_accuracy: 0.9844
Epoch 15/23
512/512 [==============================] - 2s 4ms/sample - loss: 0.0982 - 

Train on 512 samples, validate on 64 samples
Epoch 24/67
512/512 [==============================] - 2s 4ms/sample - loss: 0.8647 - accuracy: 0.9121 - val_loss: 0.0486 - val_accuracy: 0.9844
Epoch 25/67
512/512 [==============================] - 1s 3ms/sample - loss: 0.1136 - accuracy: 0.9902 - val_loss: 0.0040 - val_accuracy: 1.0000
Epoch 26/67
512/512 [==============================] - 1s 1ms/sample - loss: 0.0346 - accuracy: 0.9883 - val_loss: 0.0639 - val_accuracy: 0.9844
Epoch 27/67
512/512 [==============================] - 1s 1ms/sample - loss: 2.9549e-04 - accuracy: 1.0000 - val_loss: 0.0663 - val_accuracy: 0.9844
Epoch 28/67
512/512 [==============================] - 1s 3ms/sample - loss: 0.0174 - accuracy: 0.9961 - val_loss: 4.7159e-04 - val_accuracy: 1.0000
Epoch 29/67
512/512 [==============================] - 1s 3ms/sample - loss: 0.2419 - accuracy: 0.9941 - val_loss: 4.0061e-04 - val_accuracy: 1.0000
Epoch 30/67
512/512 [==============================] - 1s 1ms/sample - lo

Train on 512 samples, validate on 64 samples
Epoch 24/67
512/512 [==============================] - 1s 1ms/sample - loss: 0.3759 - accuracy: 0.8418 - val_loss: 0.0619 - val_accuracy: 0.9844
Epoch 25/67
512/512 [==============================] - 0s 401us/sample - loss: 0.0054 - accuracy: 0.9980 - val_loss: 0.0117 - val_accuracy: 1.0000
Epoch 26/67
512/512 [==============================] - 0s 412us/sample - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 27/67
512/512 [==============================] - 0s 449us/sample - loss: 3.8421e-04 - accuracy: 1.0000 - val_loss: 0.0064 - val_accuracy: 1.0000
Epoch 28/67
512/512 [==============================] - 0s 403us/sample - loss: 4.1153e-04 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 29/67
512/512 [==============================] - 0s 410us/sample - loss: 3.4070e-04 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 30/67
512/512 [==============================] - 0s 428us

In [16]:
tuner.get_best_hyperparameters(num_trials=10)[0].values

{'Activation': 'Swish',
 'Optimizer': 'SGD',
 'Pooling': 'None',
 'Regularization': 'Dropout',
 'Regularization=Dropout/Dropout_rate': 0.1,
 'batch_size': 16,
 'learning_rate': 0.01,
 'nLayers': 2,
 'nNeurons_0': 128,
 'nNeurons_1': 16,
 'nNeurons_2': 16,
 'tuner/bracket': 1,
 'tuner/epochs': 67,
 'tuner/initial_epoch': 0,
 'tuner/round': 0}

In [0]:
import pandas as pd

# Create a list from te dictionaries containing the hyperparameters ranked, and create a pandas dataframe
df = pd.DataFrame([record.values for record in tuner.get_best_hyperparameters(num_trials = 100)])

In [35]:
#View the first 20 records
df.head(20)

,Pooling,Activation,Regularization,nLayers,nNeurons_0,Regularization=Dropout/Dropout_rate,learning_rate,Optimizer,batch_size,nNeurons_1,nNeurons_2,tuner/epochs,tuner/initial_epoch,tuner/bracket,tuner/round,tuner/trial_id
0,None,Swish,Dropout,2,128,0.1,0.0100,SGD,16,16,16.0,67,0,1,0,NaN
1,None,LeakyReLU,Dropout,2,256,0.1,0.0010,SGD,8,64,128.0,8,0,3,0,NaN
2,None,ReLU,Dropout,2,64,0.1,0.0010,SGD,64,64,256.0,200,67,4,4,701eeeeb5f156874d1481a294571a7fe
3,None,LeakyReLU,Dropout,2,256,0.1,0.0010,SGD,8,64,128.0,67,23,3,2,7d07e95a9f8d3bc05701db77e97b3f25
4,None,LeakyReLU,Dropout,2,256,0.1,0.0010,Adam,128,16,128.0,200,0,0,0,NaN
5,None,LeakyReLU,Dropout,3,64,0.1,0.0010,SGD,32,256,128.0,67,23,4,3,9a074427be9fccae91b9e2f09004acb2
6,None,ReLU,Dropout,2,64,0.1,0.0010,SGD,64,64,256.0,67,23,4,3,d2785b40ce5f8ee04c7a372f71ff9216
7,None,LeakyReLU,Dropout,2,128,0.1,0.0010,SGD,128,8,64.0,200,67,2,2,b644f5c6143bac17cbe5a6c60173a5ba
8,None,ReLU,Dropout,3,256,0.1,0.0010,SGD,256,512,128.0,200,67,1,1,60e02ee28041f4294a68f5579071cf0d
9,None,LeakyReLU,Dropout,3,64,0.1,0.0010,SGD,32,256,128.0,200,67,4,4,21220aa3233f09ccd1908087587040cb


Based on the first 20 models, the parameters that really matter are the following:


*   Pooling, from which no pooling layer is the best
*   Dropout is a clear winner over using BatchNormalization
*   SGD with a learning rate of 0.0010 seems to be the most common

Besides these, I'm choosing the following hyperparameters trying to achive the fastest convergence time, and minimal number of parameters:

*   Swish activation
*   1 layer with 64 neurons
*   A dropout layer
*   A batch_size of 16

As I made a typo in defining the model which resulted in only having 0.1 tested as the dropout rate, I'm going to run a second run with only changing the dropout rate.


In [0]:
def build_class_model_dropout(hp):
    class_model = Sequential()
    class_model.add(InputLayer(input_shape = (6, 6, 2048)))

    class_model.add(Flatten())

    class_model.add(Dense(units = 64, activation = Swish(beta = 0.3)))
    class_model.add(Dropout(hp.Float('Dropout_rate', min_value = 0.1, max_value = 0.5, step = 0.05)))

    class_model.add(Dense(units = 6, activation = 'softmax'))

    class_model.compile(optimzer = SGD(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return class_model

In [51]:
tuner = Hyperband(
    build_class_model_dropout,
    objective='val_loss',
    max_epochs=200,
    factor=3,
    executions_per_trial=3,
    directory='/content',
    project_name='hyperopt_logs_dropoutRate')

tuner.search_space_summary()

In [52]:
tuner.search(train_bottleNeck, train_labels,
             batch_size = 16,
             shuffle = True,
             validation_data = [validate_bottleNeck, validate_labels],
             callbacks = [early_stopping])

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.2425 - accuracy: 0.9531 - val_loss: 0.0342 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 707us/sample - loss: 1.0903e-04 - accuracy: 1.0000 - val_loss: 0.0429 - val_accuracy: 0.9844
Epoch 3/3
512/512 [==============================] - 0s 704us/sample - loss: 0.0307 - accuracy: 0.9922 - val_loss: 0.0525 - val_accuracy: 0.9844
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.5200 - accuracy: 0.9512 - val_loss: 0.0296 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 896us/sample - loss: 6.7780e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 739us/sample - loss: 1.8928e-04 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Train on 512 samples, validate o

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.4671 - accuracy: 0.9316 - val_loss: 0.0095 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 0s 884us/sample - loss: 0.0314 - accuracy: 0.9922 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 794us/sample - loss: 0.0398 - accuracy: 0.9961 - val_loss: 0.1132 - val_accuracy: 0.9688
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.4218 - accuracy: 0.9375 - val_loss: 0.0286 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 871us/sample - loss: 0.0408 - accuracy: 0.9824 - val_loss: 8.2527e-04 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 776us/sample - loss: 0.0477 - accuracy: 0.9863 - val_loss: 0.0057 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 sam

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.2950 - accuracy: 0.9453 - val_loss: 0.0857 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 691us/sample - loss: 0.0324 - accuracy: 0.9922 - val_loss: 0.2147 - val_accuracy: 0.9844
Epoch 3/3
512/512 [==============================] - 1s 1ms/sample - loss: 0.0061 - accuracy: 0.9980 - val_loss: 1.3301e-04 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.2654 - accuracy: 0.9531 - val_loss: 0.0392 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 687us/sample - loss: 0.0833 - accuracy: 0.9824 - val_loss: 0.4213 - val_accuracy: 0.9375
Epoch 3/3
512/512 [==============================] - 0s 628us/sample - loss: 0.0869 - accuracy: 0.9902 - val_loss: 3.7878e-04 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 s

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.1653 - accuracy: 0.9590 - val_loss: 5.8366e-05 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 0s 737us/sample - loss: 3.8265e-05 - accuracy: 1.0000 - val_loss: 9.6121e-05 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 668us/sample - loss: 3.3357e-05 - accuracy: 1.0000 - val_loss: 2.2390e-04 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.2432 - accuracy: 0.9570 - val_loss: 0.0599 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 673us/sample - loss: 0.1031 - accuracy: 0.9863 - val_loss: 0.0058 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 663us/sample - loss: 0.0022 - accuracy: 0.9980 - val_loss: 4.6226e-04 - val_accuracy: 1.0000
Train on 512 samples

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.3981 - accuracy: 0.9414 - val_loss: 0.0245 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 946us/sample - loss: 0.1179 - accuracy: 0.9746 - val_loss: 7.7639e-04 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 1s 1ms/sample - loss: 0.0789 - accuracy: 0.9863 - val_loss: 1.1215e-05 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.4602 - accuracy: 0.9180 - val_loss: 0.0714 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 687us/sample - loss: 0.0329 - accuracy: 0.9902 - val_loss: 0.0567 - val_accuracy: 0.9844
Epoch 3/3
512/512 [==============================] - 0s 774us/sample - loss: 0.1135 - accuracy: 0.9863 - val_loss: 0.1899 - val_accuracy: 0.9844
Train on 512 samples, validate on 64 s

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.0990 - accuracy: 0.9609 - val_loss: 0.0227 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 735us/sample - loss: 0.1257 - accuracy: 0.9844 - val_loss: 0.0396 - val_accuracy: 0.9844
Epoch 3/3
512/512 [==============================] - 0s 701us/sample - loss: 0.0376 - accuracy: 0.9922 - val_loss: 0.3529 - val_accuracy: 0.9844
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.3245 - accuracy: 0.9512 - val_loss: 0.0106 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 1s 988us/sample - loss: 4.5219e-04 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 774us/sample - loss: 0.0515 - accuracy: 0.9922 - val_loss: 0.0212 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 sam

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.1938 - accuracy: 0.9414 - val_loss: 0.0113 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 0s 974us/sample - loss: 0.0137 - accuracy: 0.9941 - val_loss: 8.5527e-04 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 0s 946us/sample - loss: 0.0560 - accuracy: 0.9902 - val_loss: 1.8938e-05 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.4195 - accuracy: 0.9238 - val_loss: 9.9096e-04 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 0s 686us/sample - loss: 0.0146 - accuracy: 0.9922 - val_loss: 0.0442 - val_accuracy: 0.9844
Epoch 3/3
512/512 [==============================] - 0s 703us/sample - loss: 0.1453 - accuracy: 0.9824 - val_loss: 0.0076 - val_accuracy: 1.0000
Train on 512 samples, validate o

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.2957 - accuracy: 0.9453 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 0s 688us/sample - loss: 0.0077 - accuracy: 0.9961 - val_loss: 0.1617 - val_accuracy: 0.9688
Epoch 3/3
512/512 [==============================] - 0s 760us/sample - loss: 0.1130 - accuracy: 0.9883 - val_loss: 0.2338 - val_accuracy: 0.9844
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.4067 - accuracy: 0.9336 - val_loss: 0.0852 - val_accuracy: 0.9688
Epoch 2/3
512/512 [==============================] - 0s 681us/sample - loss: 0.0289 - accuracy: 0.9902 - val_loss: 0.0301 - val_accuracy: 0.9844
Epoch 3/3
512/512 [==============================] - 0s 693us/sample - loss: 0.0276 - accuracy: 0.9922 - val_loss: 0.1735 - val_accuracy: 0.9844
Train on 512 samples, validate on 64 samples

Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.2433 - accuracy: 0.9551 - val_loss: 0.0262 - val_accuracy: 0.9844
Epoch 2/3
512/512 [==============================] - 0s 699us/sample - loss: 0.0699 - accuracy: 0.9883 - val_loss: 0.0401 - val_accuracy: 0.9688
Epoch 3/3
512/512 [==============================] - 0s 934us/sample - loss: 0.0315 - accuracy: 0.9980 - val_loss: 0.0016 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 samples
Epoch 1/3
512/512 [==============================] - 1s 2ms/sample - loss: 0.3277 - accuracy: 0.9492 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 2/3
512/512 [==============================] - 0s 674us/sample - loss: 0.0022 - accuracy: 0.9980 - val_loss: 0.0082 - val_accuracy: 1.0000
Epoch 3/3
512/512 [==============================] - 1s 1ms/sample - loss: 0.0019 - accuracy: 0.9980 - val_loss: 5.4071e-04 - val_accuracy: 1.0000
Train on 512 samples, validate on 64 sampl

INFO:tensorflow:Oracle triggered exit


Check the results using the method seen before.

In [53]:
# Create a list from te dictionaries containing the hyperparameters ranked, and create a pandas dataframe
df = pd.DataFrame([record.values for record in tuner.get_best_hyperparameters(num_trials = 10)])
df.head(10)

,Dropout_rate,tuner/epochs,tuner/initial_epoch,tuner/bracket,tuner/round
0,0.20,3,0,4,0
1,0.40,3,0,4,0
2,0.50,3,0,4,0
3,0.30,3,0,4,0
4,0.25,3,0,4,0
5,0.35,3,0,4,0
6,0.10,3,0,4,0
7,0.45,3,0,4,0
8,0.15,3,0,4,0


By looking at the logs, it seems like dropout rate does not seem to have a huge impact. I'll chose the first result from here, which is 0.2.

*NOTE*: As for now, I couldn't find a way to see both the hyperparameters, and the metrics achieved by them. It'd be nice if one could also see in the results the following:


*   The final tracked metric(s) for the model
*   The number of parameters for the model
*   The time it took to train that model

